<a href="https://colab.research.google.com/github/ChongLC/bio-server/blob/main/Corona_SRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/saketkc/pysradb

  Cloning https://github.com/saketkc/pysradb to /tmp/pip-req-build-ijlf1zgr
  Running command git clone --filter=blob:none --quiet https://github.com/saketkc/pysradb /tmp/pip-req-build-ijlf1zgr
  Resolved https://github.com/saketkc/pysradb to commit bc7d54827903477bd59b7ec02774e6c7f41b7c3d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pysradb: filename=pysradb-2.2.2-py3-none-any.whl size=171020 sha256=82ec3a26f443fc564d149f780cd41f3938723d6883bd8ed02264171377c8c294
  Stored in directory: /tmp/pip-ephem-wheel-cache-e0mjpl5s/wheels/c7/8a/b3/97c7f3f62e0aad8b9ab34358af3b45e4793a7c93bb5deb3c09
  Created wheel for requests-ftp: filename=requests_ftp-0.3.1-py3-none-any.whl size=8164 sha256=52dd4378f00d983ecad5c0b07ebbef7de2248bd951a8485992a81a93db371273
  Stored in directory: /root/.cache/pip/wheels/c9/06/7b/2ce24ee089ed6028ebf8073740d9c35b819de

In [ ]:
from pysradb.sraweb import SRAweb

/usr/local/lib/python3.11/dist-packages/pysradb/utils.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
import pandas as pd
df_sraDB_full = pd.read_csv('SRR_list.csv')
df_sraDB_full # 883

df_sraDB = pd.read_csv('df_sraDB.csv')
df_sraDB # 63

,SRA_id_host,taxid,collection_date
0,SRR18579897,10036,missing
1,SRR15070094,10036,2021-03
2,SRR15070102,10036,2021-03
3,SRR8427114,94323,2017
4,SRR8335009,9030,2017-04-18
...,...,...,...
58,SRR15070105,10036,2021-03
59,SRR18579898,10036,missing
60,SRR10708182,496922,2018-02-13
61,SRR8329346,431241,2017-11-10


In [ ]:
previous_sraDB = pd.read_csv('sra_hostLineage_table.csv')
previous_sraDB = previous_sraDB[['SRA_id', 'taxid']]
previous_sraDB = previous_sraDB.drop_duplicates().reset_index(drop=True)
previous_sraDB

,SRA_id,taxid
0,SRR6486648,2070543.0
1,SRR10530194,199225.0
2,SRR10530195,199225.0
3,SRR10530196,199225.0
4,SRR10530197,199225.0
...,...,...
650,SRR8082050,148545.0
651,SRR8082051,148545.0
652,SRR5405109,3656.0
653,SRR2777577,9913.0


In [ ]:
comparison_df = pd.merge(df_sraDB_full, df_sraDB, on='SRA_id_host', how='left')
comparison_df

comparison_df = comparison_df.merge(previous_sraDB[['SRA_id', 'taxid']],
                       left_on='SRA_id_host',
                       right_on='SRA_id',
                       how='left')
comparison_df['taxid'] = comparison_df['taxid_x'].combine_first(comparison_df['taxid_y'])
comparison_df.drop(columns=['SRA_id', 'taxid_x', 'taxid_y'], inplace=True)
comparison_df

nan_df = comparison_df[comparison_df['taxid'].isna()]
nan_df = nan_df[nan_df['SRA_id_host'].str.startswith(('SRR', 'DRR', 'ERR'), na=False)]
nan_df

,contig_id,SRA_id_host,collection_date,taxid
0,SRR22582176_NODE_2_length_21742_cov_457.590815,SRR22582176,NaN,NaN
4,SRR13451125_NODE_1_length_10902_cov_179.219323,SRR13451125,NaN,NaN
5,SRR17844015_NODE_1_length_21584_cov_225.397882,SRR17844015,NaN,NaN
6,SRR14785495_NODE_10_length_21584_cov_157.220493,SRR14785495,NaN,NaN
7,SRR18155281_NODE_1_length_21356_cov_52.721609,SRR18155281,NaN,NaN
...,...,...,...,...
871,SRR719255_Supercontig_6086,SRR719255,NaN,NaN
872,SRR13989988_NODE_2_length_20337_cov_1351.432107,SRR13989988,NaN,NaN
873,SRR16151766_NODE_6320_length_2762_cov_3.729221,SRR16151766,NaN,NaN
875,SRR15068955_NODE_57_length_11890_cov_56.113815,SRR15068955,NaN,NaN


In [ ]:
nan_df = nan_df[['contig_id', 'SRA_id_host']]
nan_df.to_csv('unknown.csv', index = False)

nan_df

,contig_id,SRA_id_host
0,SRR22582176_NODE_2_length_21742_cov_457.590815,SRR22582176
4,SRR13451125_NODE_1_length_10902_cov_179.219323,SRR13451125
5,SRR17844015_NODE_1_length_21584_cov_225.397882,SRR17844015
6,SRR14785495_NODE_10_length_21584_cov_157.220493,SRR14785495
7,SRR18155281_NODE_1_length_21356_cov_52.721609,SRR18155281
...,...,...
871,SRR719255_Supercontig_6086,SRR719255
872,SRR13989988_NODE_2_length_20337_cov_1351.432107,SRR13989988
873,SRR16151766_NODE_6320_length_2762_cov_3.729221,SRR16151766
875,SRR15068955_NODE_57_length_11890_cov_56.113815,SRR15068955


In [ ]:
unknown = nan_df.SRA_id_host.tolist()
unknown

['SRR22582176',
 'SRR13451125',
 'SRR17844015',
 'SRR14785495',
 'SRR18155281',
 'SRR17849665',
 'SRR13451137',
 'SRR15663690',
 'SRR18989502',
 'SRR18155299',
 'SRR13451211',
 'SRR13753889',
 'SRR19537439',
 'SRR17249882',
 'SRR18057834',
 'SRR12558897',
 'SRR14917181',
 'SRR13302930',
 'SRR17591859',
 'ERR5324689',
 'SRR14579873',
 'SRR17249844',
 'SRR15663682',
 'ERR5324682',
 'SRR13451130',
 'SRR13753885',
 'SRR18298792',
 'SRR15093453',
 'SRR13451152',
 'SRR13451128',
 'SRR15663685',
 'SRR12544777',
 'SRR15291583',
 'SRR19206976',
 'SRR18989501',
 'SRR15291571',
 'SRR13451131',
 'SRR17844002',
 'SRR10053322',
 'SRR15291566',
 'SRR15615962',
 'SRR22582166',
 'SRR13451126',
 'SRR13402577',
 'SRR14785483',
 'SRR13451126',
 'SRR14785485',
 'SRR14785492',
 'SRR19206976',
 'SRR10574298',
 'SRR15914618',
 'SRR12538758',
 'SRR16219611',
 'SRR13451147',
 'SRR16151772',
 'SRR18155302',
 'SRR13753892',
 'SRR17188785',
 'SRR18989499',
 'SRR15663694',
 'SRR13753888',
 'SRR15291569',
 'SRR17844

In [ ]:
import pysradb
import pandas as pd

# Initialize SRAweb
db_sra = pysradb.SRAweb()

# Split the unknown list into chunks of 50
chunk_size = 50
chunks = [unknown[i:i + chunk_size] for i in range(0, len(unknown), chunk_size)]

# Store results
all_results = []

# Loop through each chunk and fetch metadata
for chunk in chunks:
    try:
        res = db_sra.sra_metadata(chunk, detailed=True)

        # Ensure res is not empty
        if res is not None and not res.empty:
            # Standardize columns (ensure missing ones are filled with None)
            expected_columns = ["run_accession", "organism_taxid", "collection_date"]  # Adjust based on your needs
            for col in expected_columns:
                if col not in res.columns:
                    res[col] = None  # Add missing columns with None

            # Keep only required columns & reset index
            res = res[expected_columns]
            res = res.reset_index(drop=True)

            all_results.append(res)

    except Exception as e:
        print(f"Error processing chunk: {chunk}\nError: {e}")

# Combine all results into a single DataFrame
if all_results:
    final_res = pd.concat(all_results, axis=0, ignore_index=True)
    print(final_res)
else:
    print("No data retrieved.")

     run_accession organism_taxid collection_date
0       ERR5324682          10036            None
1       ERR5324689          10036            None
2      SRR10053322           9823            None
3      SRR10574298           9534            None
4      SRR12544777           9685            None
...            ...            ...             ...
1151   SRR18587412           9823            None
1152   SRR18676290          60711            None
1153   SRR19206965          10036            None
1154   SRR20272752          10116            None
1155     SRR719255         175694            None

[1156 rows x 3 columns]


In [ ]:
final_res = final_res.drop_duplicates()

merged_df = nan_df.merge(final_res,
                         left_on='SRA_id_host',
                         right_on='run_accession',
                         how='left')

merged_df.drop(columns=['run_accession'], inplace=True)
merged_df # next merge with comparison_df

,contig_id,SRA_id_host,organism_taxid,collection_date
0,SRR22582176_NODE_2_length_21742_cov_457.590815,SRR22582176,10036,None
1,SRR13451125_NODE_1_length_10902_cov_179.219323,SRR13451125,392815,None
2,SRR17844015_NODE_1_length_21584_cov_225.397882,SRR17844015,10036,None
3,SRR14785495_NODE_10_length_21584_cov_157.220493,SRR14785495,10036,None
4,SRR18155281_NODE_1_length_21356_cov_52.721609,SRR18155281,9534,None
...,...,...,...,...
581,SRR719255_Supercontig_6086,SRR719255,175694,None
582,SRR13989988_NODE_2_length_20337_cov_1351.432107,SRR13989988,9823,None
583,SRR16151766_NODE_6320_length_2762_cov_3.729221,SRR16151766,9544,None
584,SRR15068955_NODE_57_length_11890_cov_56.113815,SRR15068955,10036,None


In [ ]:
comparison_df

,contig_id,SRA_id_host,collection_date,taxid
0,SRR22582176_NODE_2_length_21742_cov_457.590815,SRR22582176,NaN,NaN
1,SRR1195277_Supercontig_1,SRR1195277,NaN,9534.0
2,SRR15070094_NODE_1_length_28255_cov_103.902411,SRR15070094,2021-03,10036.0
3,SRR8835217_Supercontig_1,SRR8835217,NaN,4081.0
4,SRR13451125_NODE_1_length_10902_cov_179.219323,SRR13451125,NaN,NaN
...,...,...,...,...
886,Paralichthys_olivaceus_coronavirus_segment1,Paralichthys_olivaceus,NaN,NaN
887,Periophthalmodon_schlosseri_coronavirus_segment1,Periophthalmodon_schlosseri,NaN,NaN
888,Hypophthalmichthys_nobilis_coronavirus_segment1,Hypophthalmichthys_nobilis,NaN,NaN
889,Takifugu_pardalis_coronavirus_segment1,Takifugu_pardalis,NaN,NaN


In [ ]:
import numpy as np

final_df = comparison_df.merge(merged_df,
                               on=['contig_id', 'SRA_id_host'],
                               how='left')

# Ensure 'taxid' column is updated from 'organism_taxid' and drop 'organism_taxid'
if 'organism_taxid' in final_df.columns:
    final_df['taxid'] = final_df['taxid'].combine_first(final_df['organism_taxid'])
    final_df.drop(columns=['organism_taxid'], inplace=True)

# Merge collection_date columns if both exist
if 'collection_date_x' in final_df.columns and 'collection_date_y' in final_df.columns:
    final_df['collection_date'] = final_df['collection_date_x'].combine_first(final_df['collection_date_y'])
    final_df.drop(columns=['collection_date_x', 'collection_date_y'], inplace=True)

final_df['collection_date'] = final_df['collection_date'].replace(['None', 'none', 'missing', '', pd.NA], np.nan)
final_df['taxid'] = pd.to_numeric(final_df['taxid'], errors='coerce')
final_df = final_df.drop_duplicates()
final_df

,contig_id,SRA_id_host,taxid,collection_date
0,SRR22582176_NODE_2_length_21742_cov_457.590815,SRR22582176,10036.0,NaN
1,SRR1195277_Supercontig_1,SRR1195277,9534.0,NaN
2,SRR15070094_NODE_1_length_28255_cov_103.902411,SRR15070094,10036.0,2021-03
3,SRR8835217_Supercontig_1,SRR8835217,4081.0,NaN
4,SRR13451125_NODE_1_length_10902_cov_179.219323,SRR13451125,392815.0,NaN
...,...,...,...,...
886,Paralichthys_olivaceus_coronavirus_segment1,Paralichthys_olivaceus,NaN,NaN
887,Periophthalmodon_schlosseri_coronavirus_segment1,Periophthalmodon_schlosseri,NaN,NaN
888,Hypophthalmichthys_nobilis_coronavirus_segment1,Hypophthalmichthys_nobilis,NaN,NaN
889,Takifugu_pardalis_coronavirus_segment1,Takifugu_pardalis,NaN,NaN


In [ ]:
final_df.to_csv('v3_df_sraDB.csv', index = False)

# txid

In [ ]:
!pip install ete3
from ete3 import NCBITaxa

ncbi = NCBITaxa()

wanted_taxid = final_df.taxid.dropna().astype(int).tolist()
results = list()
desired_ranks = desired_ranks = ['superkingdom', 'clade', 'kingdom', 'subkingdom', 'superphylum', 'phylum', 'subphylum', 'superclass', 'class', 'subclass', 'superorder', 'order', 'suborder', 'superfamily', 'family', 'subfamily', 'genus', 'subgenus', 'species']

def get_ranks(taxid, desired_ranks):
    lineage = ncbi.get_lineage(taxid)
    names = ncbi.get_taxid_translator(lineage)
    lineage2ranks = ncbi.get_rank(names)
    ranks2lineage = dict((rank,taxid) for (taxid, rank) in lineage2ranks.items())
    return{'{}_id'.format(rank): ranks2lineage.get(rank, '<not present>') for rank in desired_ranks}

for taxid in wanted_taxid:
  results.append(list())
  results[-1].append(str(taxid))
  ranks = get_ranks(taxid, desired_ranks)
  for key, rank in ranks.items():
    if rank != '<not present>':
      results[-1].append(list(ncbi.get_taxid_translator([rank]).values())[0])
    else:
      results[-1].append(rank)

i = 0
txid2lineage = open('txid2lineage_final.tsv','a')
for result in results:
    txid2lineage.write('\t'.join(result) + '\n')

    i += 1

txid2lineage.close()

In [ ]:
taxonomy_file = 'txid2lineage_final.tsv'
tax_data = pd.read_csv(taxonomy_file, sep='\t', names=['taxid', 'superkingdom', 'clade', 'kingdom', 'subkingdom', 'superphylum', 'phylum', 'subphylum', 'superclass', 'class', 'subclass', 'superorder', 'order', 'suborder', 'superfamily', 'family', 'subfamily', 'genus', 'subgenus', 'species'])
tax_data.drop_duplicates(keep='first', inplace=True)
tax_data # 69

,taxid,superkingdom,clade,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,superorder,order,suborder,superfamily,family,subfamily,genus,subgenus,species
0,10036,<not present>,Boreoeutheria,Metazoa,<not present>,<not present>,Chordata,Craniata,Sarcopterygii,Mammalia,<not present>,Euarchontoglires,Rodentia,Myomorpha,<not present>,Cricetidae,Cricetinae,Mesocricetus,<not present>,Mesocricetus auratus
1,9534,<not present>,Boreoeutheria,Metazoa,<not present>,<not present>,Chordata,Craniata,Sarcopterygii,Mammalia,<not present>,Euarchontoglires,Primates,Haplorrhini,Cercopithecoidea,Cercopithecidae,Cercopithecinae,Chlorocebus,<not present>,Chlorocebus aethiops
3,4081,<not present>,Pentapetalae,Viridiplantae,<not present>,<not present>,Streptophyta,Streptophytina,<not present>,Magnoliopsida,<not present>,<not present>,Solanales,<not present>,<not present>,Solanaceae,Solanoideae,Solanum,Solanum subgen. Lycopersicon,Solanum lycopersicum
4,392815,<not present>,Boreoeutheria,Metazoa,<not present>,<not present>,Chordata,Craniata,Sarcopterygii,Mammalia,<not present>,Euarchontoglires,Primates,Haplorrhini,Cercopithecoidea,Cercopithecidae,Cercopithecinae,Chlorocebus,<not present>,<not present>
8,9823,<not present>,Boreoeutheria,Metazoa,<not present>,<not present>,Chordata,Craniata,Sarcopterygii,Mammalia,<not present>,Laurasiatheria,Artiodactyla,Suina,<not present>,Suidae,<not present>,Sus,<not present>,Sus scrofa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,436086,<not present>,Pentapetalae,Viridiplantae,<not present>,<not present>,Streptophyta,Streptophytina,<not present>,Magnoliopsida,<not present>,<not present>,Rosales,<not present>,<not present>,Rosaceae,Amygdaloideae,Pyrus,<not present>,Pyrus betulifolia
856,79674,<not present>,Ecdysozoa,Metazoa,<not present>,<not present>,Arthropoda,Crustacea,Multicrustacea,Malacostraca,Eumalacostraca,Eucarida,Decapoda,Pleocyemata,Palaemonoidea,Palaemonidae,<not present>,Macrobrachium,<not present>,Macrobrachium rosenbergii
860,46731,<not present>,Opisthokonta,Metazoa,<not present>,<not present>,Cnidaria,<not present>,<not present>,Anthozoa,Hexacorallia,<not present>,Scleractinia,Astrocoeniina,<not present>,Pocilloporidae,<not present>,Pocillopora,<not present>,Pocillopora damicornis
862,9541,<not present>,Boreoeutheria,Metazoa,<not present>,<not present>,Chordata,Craniata,Sarcopterygii,Mammalia,<not present>,Euarchontoglires,Primates,Haplorrhini,Cercopithecoidea,Cercopithecidae,Cercopithecinae,Macaca,<not present>,Macaca fascicularis


In [ ]:
full_tax_data = pd.merge(final_df, tax_data, on='taxid', how='left')
full_tax_data.drop_duplicates(keep='first', inplace=True)
full_tax_data # 883

,contig_id,SRA_id_host,taxid,collection_date,superkingdom,clade,kingdom,subkingdom,superphylum,phylum,...,subclass,superorder,order,suborder,superfamily,family,subfamily,genus,subgenus,species
0,SRR22582176_NODE_2_length_21742_cov_457.590815,SRR22582176,10036.0,NaN,<not present>,Boreoeutheria,Metazoa,<not present>,<not present>,Chordata,...,<not present>,Euarchontoglires,Rodentia,Myomorpha,<not present>,Cricetidae,Cricetinae,Mesocricetus,<not present>,Mesocricetus auratus
1,SRR1195277_Supercontig_1,SRR1195277,9534.0,NaN,<not present>,Boreoeutheria,Metazoa,<not present>,<not present>,Chordata,...,<not present>,Euarchontoglires,Primates,Haplorrhini,Cercopithecoidea,Cercopithecidae,Cercopithecinae,Chlorocebus,<not present>,Chlorocebus aethiops
2,SRR15070094_NODE_1_length_28255_cov_103.902411,SRR15070094,10036.0,2021-03,<not present>,Boreoeutheria,Metazoa,<not present>,<not present>,Chordata,...,<not present>,Euarchontoglires,Rodentia,Myomorpha,<not present>,Cricetidae,Cricetinae,Mesocricetus,<not present>,Mesocricetus auratus
3,SRR8835217_Supercontig_1,SRR8835217,4081.0,NaN,<not present>,Pentapetalae,Viridiplantae,<not present>,<not present>,Streptophyta,...,<not present>,<not present>,Solanales,<not present>,<not present>,Solanaceae,Solanoideae,Solanum,Solanum subgen. Lycopersicon,Solanum lycopersicum
4,SRR13451125_NODE_1_length_10902_cov_179.219323,SRR13451125,392815.0,NaN,<not present>,Boreoeutheria,Metazoa,<not present>,<not present>,Chordata,...,<not present>,Euarchontoglires,Primates,Haplorrhini,Cercopithecoidea,Cercopithecidae,Cercopithecinae,Chlorocebus,<not present>,<not present>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,Paralichthys_olivaceus_coronavirus_segment1,Paralichthys_olivaceus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
879,Periophthalmodon_schlosseri_coronavirus_segment1,Periophthalmodon_schlosseri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
880,Hypophthalmichthys_nobilis_coronavirus_segment1,Hypophthalmichthys_nobilis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
881,Takifugu_pardalis_coronavirus_segment1,Takifugu_pardalis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
full_tax_data.to_csv('corona_sra_full_host_lineage.csv', index=False)